In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from typing import TypedDict
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent, AgentState
from langchain.tools import tool 
import os
from langchain.agents.middleware import wrap_tool_call, dynamic_prompt, ModelRequest
from langchain_core.messages import ToolMessage
from pydantic import BaseModel, Field
from typing import Optional
import pandas as pd
from search_food import RAGSystem, Text
#from my_tools import get_recipes_by_ingredients


In [3]:
txt = Text("saved_txt.txt")
rag = RAGSystem(txt)

Векторная БД загружена из файла


In [4]:

kk = pd.read_excel("kk.xlsx")

def parse_range(value):
    """Преобразует строку с диапазоном в среднее число"""
    if pd.isna(value):
        return None
    try:
        # Заменяем запятые на точки и разделяем по дефису
        parts = str(value).replace(',', '.').split('-')
        # Убираем пробелы и преобразуем в числа
        numbers = [float(part.strip()) for part in parts]
        # Возвращаем среднее значение
        return sum(numbers) / len(numbers)
    except:
        return None

# Применяем ко всем столбцам с диапазонами
kk['squirrels_avg'] = kk['squirrels'].apply(parse_range)
kk['fats_avg'] = kk['fats'].apply(parse_range)
kk['carbohydrates_avg'] = kk['carbohydrates'].apply(parse_range)
kk['callories_avg'] = kk['callories'].apply(parse_range)
kk

,Product,squirrels,fats,carbohydrates,callories,squirrels_avg,fats_avg,carbohydrates_avg,callories_avg
0,Хлеб пшеничный из муки высшего сорта,"7,5-7,6","0,8-2,9","49,2-51,4",235-262,7.55,1.85,50.3,248.5
1,Хлеб пшеничный из муки 1 сорта,7.6,0.9,49.7,226,7.60,0.90,49.7,226.0
2,Хлеб из ржано-пшеничной муки,6.8,1.2,46.4,215,6.80,1.20,46.4,215.0
3,Хлеб ржаной,4.7,0.7,49.8,214,4.70,0.70,49.8,214.0
4,"Хлеб ""Бородинский""",6.9,1.3,40.9,208,6.90,1.30,40.9,208.0
...,...,...,...,...,...,...,...,...,...
631,Саке,0.5,-,5,134,0.50,NaN,5.0,134.0
632,Самогон,0.1,0.1,0.4,235,0.10,0.10,0.4,235.0
633,Спирт этиловый 96%,4,-,3.8,710,4.00,NaN,3.8,710.0
634,Текила,1.4,0.3,24,231,1.40,0.30,24.0,231.0


In [5]:
kk[kk["Product"] == 'Сыр "Гауда"']

,Product,squirrels,fats,carbohydrates,callories,squirrels_avg,fats_avg,carbohydrates_avg,callories_avg
96,"Сыр ""Гауда""",25,27,2,356,25.0,27.0,2.0,356.0


In [6]:
class ProductSearch(BaseModel):
    """Ввод данных для запросов о еде"""

    name: Optional[str] = Field(description="Название продукта", default=None)
    squirrels: Optional[str] = Field(description="Белки", default=None)
    fats: Optional[str] = Field(description="Жиры", default=None)
    carbohydrates: Optional[str] = Field(description="Углеводы", default=None)
    callories: Optional[str] = Field(description="Энергия", default=None)
    squirrels_avg: Optional[str] = Field(description="Белки среднее значение", default=None)
    fats_avg: Optional[str] = Field(description="Жиры среднее значение", default=None)
    carbohydrates_avg: Optional[str] = Field(description="Углеводы среднее значение", default=None)
    callories_avg: Optional[str] = Field(description="Энергия среднее значение", default=None)
    
    # Добавляем параметры для диапазонного поиска
    min_squirrels: Optional[int] = Field(description="Минимальное количество белков", default=None)
    max_squirrels: Optional[int] = Field(description="Максимальное количество белков", default=None)
    min_fats: Optional[int] = Field(description="Минимальное количество жиров", default=None)
    max_fats: Optional[int] = Field(description="Максимальное количество жиров", default=None)
    min_carbohydrates: Optional[int] = Field(description="Минимальное количество углеводов", default=None)
    max_carbohydrates: Optional[int] = Field(description="Максимальное количество углеводов", default=None)
    min_callories: Optional[int] = Field(description="Минимальная энергетическая ценность", default=None)
    max_callories: Optional[int] = Field(description="Максимальная энергетическая ценность", default=None)
    number_products: Optional[int] = Field(description="Количество продуктов", default=None)
    
    sort_order: Optional[str] = Field(
        description="Порядок сортировки (наибольшие белки, наименьшие белки, наибольшие жиры, наименьшие жиры, наибольшие углеводы, наименьшие углеводы,наибольшие каллории, наименьшие каллории)",
        default=None,
    )

In [7]:
@tool(args_schema=ProductSearch)
def search_food(
    name: Optional[str] = None,
    squirrels: Optional[str] = None,
    fats: Optional[str] = None,
    carbohydrates: Optional[str] = None,
    callories: Optional[str] = None,
    squirrels_avg: Optional[str] = None,
    fats_avg: Optional[str] = None,
    carbohydrates_avg: Optional[str] = None,
    callories_avg: Optional[str] = None,
    min_squirrels: Optional[int] = None,
    max_squirrels: Optional[int] = None,
    min_fats: Optional[int] = None,
    max_fats: Optional[int] = None,
    min_carbohydrates: Optional[int] = None,
    max_carbohydrates: Optional[int] = None,
    min_callories: Optional[int] = None,
    max_callories: Optional[int] = None,
    sort_order: Optional[str] = None,
    number_products: Optional[int] = None
) -> str:
    """Поиск продуктов питания по различным критериям питания"""
    
    x = kk.copy()
    
    # Apply filters
    if name:  
        x = x[x["Product"].apply(lambda x: name.lower() in str(x).lower())]
    if min_callories: 
        x = x[x["callories_avg"] >= min_callories]
    if max_callories: 
        x = x[x["callories_avg"] <= max_callories]
    if min_carbohydrates: 
        x = x[x["carbohydrates_avg"] >= min_carbohydrates]
    if max_carbohydrates: 
        x = x[x["carbohydrates_avg"] <= max_carbohydrates]
    if min_fats: 
        x = x[x["fats_avg"] >= min_fats]
    if max_fats: 
        x = x[x["fats_avg"] <= max_fats]
    if min_squirrels: 
        x = x[x["squirrels_avg"] >= min_squirrels]
    if max_squirrels: 
        x = x[x["squirrels_avg"] <= max_squirrels]
    if sort_order:
        if sort_order == "наибольшие белки":x = x.sort_values('squirrels_avg', ascending=False)
        if sort_order == "наименьшие белки":x = x.sort_values('squirrels_avg', ascending=True)
        if sort_order == "наибольшие жиры":x = x.sort_values('fats_avg', ascending=False)
        if sort_order == "наименьшие жиры":x = x.sort_values('fats_avg', ascending=True)
        if sort_order == "наибольшие углеводы":x = x.sort_values('carbohydrates_avg', ascending=False)
        if sort_order == "наименьшие углеводы":x = x.sort_values('carbohydrates_avg', ascending=True)
        if sort_order == "наибольшие каллории":x = x.sort_values('callories_avg', ascending=False)
        if sort_order == "наименьшие каллории":x = x.sort_values('callories_avg', ascending=True)
    if number_products: x = x.head(number_products)
    # Формируем результат
    result = "Результаты поиска продуктов:\n"
    
    for i, (idx, food) in enumerate(x.iterrows(), 1):
        result += f"{i}. {food['Product']} - Б: {food.get('squirrels_avg', 'N/A')}г, Ж: {food.get('fats_avg', 'N/A')}г, У: {food.get('carbohydrates_avg', 'N/A')}г, Кал: {food.get('callories_avg', 'N/A')}\n"
    
    result += f"\nНайдено продуктов: {len(x)}"
    
    return result

In [8]:
carts = {}

class AddToCart(BaseModel):
    """Ввод данных для добавления продукта в корзину"""
    name: str = Field(description="Точное название еды, чтобы положить в корзину", default=None)
    count: float = Field(description="Количество, которое нужно положить в корзину. Измеряется в граммах",default=100.0)

@tool(args_schema=AddToCart)
def add_product_to_cart(
    name: Optional[str] = None,
    count: Optional[float] = 100.0
    ):
    """Добавление продуктов в коризну carts"""
    
    if name not in kk['Product'].values:
        return f"Продукт '{name}' не найден. Пожалуйста, проверьте название."
    
    # Добавляем в корзину
    carts[name] = carts.get(name, 0) + count
    
    return f'{count} грамм "{name}" добавлено в корзину. Теперь в корзине: {carts[name]} грамм'


In [9]:
@tool(description="Показывает содержимое корзины")
def show_product_in_cart():
    """Показать продукты в корзине carts"""
    
    return carts

In [10]:
base_prompt = """Ты полезный ассистент по питанию. Ты получил результаты поиска продуктов. 
            Сформируй красивый, читаемый ответ для пользователя на русском языке.

            Если нужен список продуктов вот структурированный ответ:
            1. Краткое введение
            2. Группируй продукты по категориям (мясо, рыба, молочные и т.д.)
            3. Для каждого продукта укажи ключевые показатели
            4. Дай рекомендации

            Используй эмодзи и понятное форматирование."""

In [11]:
@tool
def search_dish(question: str) -> str:
    """Искать информацию о приготовлении блюда и продуктах для него"""
    answer = rag.ask_question(question)
    return answer

Еще инструменты

In [12]:
@tool(description="Удаляет один продукт из корзины")
def remove_from_cart(product_name: str, amount: Optional[float] = None):
    """Удалить продукт из корзины полностью или частично, если указаны граммы"""
    if amount:
        carts[product_name] -= amount
        if carts[product_name] <=0: del carts[product_name]
    else: del carts[product_name]
        

@tool(description="Удаляет абсолютно все продукты из корзины")
def clear_cart():
    """Удалить все продукты из корзины"""
    carts = {}

@tool
def calculate_cart_nutrition():
    """Посчитать общую nutritional ценность корзины"""
    # Сумма белков/жиров/углеводов/калорий всех продуктов в корзине

In [ ]:
llm = ChatOpenAI(
    model="Qwen/Qwen3-235B-A22B-Instruct-2507",
    openai_api_key=os.environ.get("GIGA"),
    openai_api_base="https://foundation-models.api.cloud.ru/v1",
    max_tokens=1000
)


@tool
def get_weather(location: str) -> str:
    """Получить информацию о погоде для местоположения."""
    return f"Погода в {location}: Солнечно, 22°C"

@wrap_tool_call
def handle_tool_errors(request, handler):
    """Обработка ошибок выполнения инструментов с пользовательскими сообщениями."""
    try:
        return handler(request)
    except Exception as e:
        # Возвращаем пользовательское сообщение об ошибке модели
        return ToolMessage(
            content=f"Ошибка инструмента: Пожалуйста, проверьте ваш ввод и попробуйте снова. ({str(e)})",
            tool_call_id=request.tool_call["id"]
        )

class Context(TypedDict):
    user_role: str

@dynamic_prompt
def user_role_prompt(request: ModelRequest) -> str:
    """Генерация системного промпта на основе роли пользователя."""
    user_role = request.runtime.context.get("user_role", "user")
    

    if user_role == "expert":
        return f"{base_prompt} Предоставляйте подробные технические ответы."
    elif user_role == "beginner":
        return f"{base_prompt} Объясняйте концепции просто и избегайте профессионального жаргона."

    return base_prompt

agent = create_agent(
    llm,
    tools=[get_weather,search_food,add_product_to_cart,show_product_in_cart,search_dish,remove_from_cart,clear_cart],
    middleware=[user_role_prompt,handle_tool_errors],
    context_schema=Context
)

# Системный промпт будет устанавливаться динамически на основе контекста


In [17]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Посоветуй блюдо,в котором есть сыр, картофель"}]},
    context={"user_role": "expert"}
)
result

{'messages': [HumanMessage(content='Посоветуй блюдо,в котором есть сыр, картофель', additional_kwargs={}, response_metadata={}, id='52c939ce-3b67-4b79-ac17-c9c2e43f4ec9'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 1638, 'total_tokens': 1670, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 1616}}, 'model_provider': 'openai', 'model_name': 'Qwen/Qwen3-235B-A22B-Instruct-2507', 'system_fingerprint': None, 'id': 'chatcmpl-f0ff6868-6603-4e04-9224-1c583cb199a0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--0d16d6da-d2bb-461b-a279-f31769bd7add-0', tool_calls=[{'name': 'search_dish', 'args': {'question': 'блюдо с сыром и картофелем'}, 'id': 'chatcmpl-tool-034a393fde2545e193ab5a6d7dba801c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1638, 'output_tokens': 32, 'total_tokens': 1670, 'input_token_details': {'cache_read': 16

In [16]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Посоветуй 3 продукта, где больше 25 белков"}]},
    context={"user_role": "expert"}
)
result

{'messages': [HumanMessage(content='Посоветуй 3 продукта, где больше 25 белков', additional_kwargs={}, response_metadata={}, id='ebdded1f-f172-4b1f-bc63-acc3fa519302'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 1637, 'total_tokens': 1668, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'Qwen/Qwen3-235B-A22B-Instruct-2507', 'system_fingerprint': None, 'id': 'chatcmpl-b420fcf6-ceb3-43df-82b8-c4c251925abe', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--6abcbc4b-3277-4d83-8894-45123bab1de2-0', tool_calls=[{'name': 'search_food', 'args': {'min_squirrels': 25, 'number_products': 3}, 'id': 'chatcmpl-tool-d0dea81e96bd42078b34d73ed0cfe703', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1637, 'output_tokens': 31, 'total_tokens': 1668, 'input_token_details': {}, 'output_token_details': {}}),
  ToolMessage(content='Рез

In [18]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Что можно приготовить с сыром?"}]},
    context={"user_role": "expert"}
)
result

{'messages': [HumanMessage(content='Что можно приготовить с сыром?', additional_kwargs={}, response_metadata={}, id='adc296c3-7d74-4682-a3b6-67db9d1f360b'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 1545, 'total_tokens': 1571, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 1520}}, 'model_provider': 'openai', 'model_name': 'Qwen/Qwen3-235B-A22B-Instruct-2507', 'system_fingerprint': None, 'id': 'chatcmpl-06be4b45-a8e8-4601-8b62-570ec44fde1d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--19a653eb-500f-410f-9996-79c9baf23664-0', tool_calls=[{'name': 'search_dish', 'args': {'question': 'блюда с сыром'}, 'id': 'chatcmpl-tool-e9ce378d223d41eb989aec47d071d38f', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1545, 'output_tokens': 26, 'total_tokens': 1571, 'input_token_details': {'cache_read': 1520}, 'output_token_details':

In [19]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": 'Добавь в корзину Сыр "Гауда" (Гауда должен быть в таких кавыках), 500 грамм'}]},
    context={"user_role": "expert"}
)

In [20]:
result

{'messages': [HumanMessage(content='Добавь в корзину Сыр "Гауда" (Гауда должен быть в таких кавыках), 500 грамм', additional_kwargs={}, response_metadata={}, id='8ed8f413-af3b-4805-96e1-d0399618bf99'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 1572, 'total_tokens': 1609, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 1520}}, 'model_provider': 'openai', 'model_name': 'Qwen/Qwen3-235B-A22B-Instruct-2507', 'system_fingerprint': None, 'id': 'chatcmpl-222d7f90-ea84-42f5-bd63-668521c28d6c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--2d3470ab-4bb4-471c-ab85-61c275983f4d-0', tool_calls=[{'name': 'add_product_to_cart', 'args': {'name': 'Сыр "Гауда"', 'count': 500}, 'id': 'chatcmpl-tool-cb12b703a9144e11becfcda406409d00', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1572, 'output_tokens': 37, 'total_tokens': 1609, 'input_

In [21]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Напиши, какой вид хлеба есть в базе?"}]},
    context={"user_role": "expert"}
)

In [22]:
result

{'messages': [HumanMessage(content='Напиши, какой вид хлеба есть в базе?', additional_kwargs={}, response_metadata={}, id='512d3fa5-722f-41ca-af24-65c17de09bf7'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 1551, 'total_tokens': 1573, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 1520}}, 'model_provider': 'openai', 'model_name': 'Qwen/Qwen3-235B-A22B-Instruct-2507', 'system_fingerprint': None, 'id': 'chatcmpl-538edc68-c220-4f60-926f-4d9fb39db21c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--d8618f04-474a-4e7a-8df0-5de37de0e8b6-0', tool_calls=[{'name': 'search_food', 'args': {'name': 'хлеб'}, 'id': 'chatcmpl-tool-21951dd433f945e99cb0063d4f13415b', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1551, 'output_tokens': 22, 'total_tokens': 1573, 'input_token_details': {'cache_read': 1520}, 'output_token_details': {}}),


In [23]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Добавь в корзину Хлеб пшеничный из муки 1 сорта 300 грамм, Ветчина рубленая 400 грамм,Масло сливочное 300 грамм"}]},
    context={"user_role": "expert"}
)
result


{'messages': [HumanMessage(content='Добавь в корзину Хлеб пшеничный из муки 1 сорта 300 грамм, Ветчина рубленая 400 грамм,Масло сливочное 300 грамм', additional_kwargs={}, response_metadata={}, id='9e81dbdd-804a-4895-b524-bced7c411490'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 117, 'prompt_tokens': 1595, 'total_tokens': 1712, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 1536}}, 'model_provider': 'openai', 'model_name': 'Qwen/Qwen3-235B-A22B-Instruct-2507', 'system_fingerprint': None, 'id': 'chatcmpl-ad75bc3e-6091-41eb-8b62-df78c8fce190', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--68006f51-8ef0-40e4-81d0-e91fc878578e-0', tool_calls=[{'name': 'add_product_to_cart', 'args': {'name': 'Хлеб пшеничный из муки 1 сорта', 'count': 300}, 'id': 'chatcmpl-tool-447c944a16ee4c14a4dc3e1df33871f8', 'type': 'tool_call'}, {'name': 'add_product_to_cart', '

In [24]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Покажи, что в корзине"}]},
    context={"user_role": "expert"}
)
result

{'messages': [HumanMessage(content='Покажи, что в корзине', additional_kwargs={}, response_metadata={}, id='d2bfb2be-fe20-440f-b972-3dfa6be6f086'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 1546, 'total_tokens': 1564, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 1520}}, 'model_provider': 'openai', 'model_name': 'Qwen/Qwen3-235B-A22B-Instruct-2507', 'system_fingerprint': None, 'id': 'chatcmpl-c5ec340c-0401-497b-982b-3234166d16ae', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--8ff98e44-4d58-433e-9502-fa697b51c485-0', tool_calls=[{'name': 'show_product_in_cart', 'args': {}, 'id': 'chatcmpl-tool-3c2c5bff93194727996d105a9232466d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1546, 'output_tokens': 18, 'total_tokens': 1564, 'input_token_details': {'cache_read': 1520}, 'output_token_details': {}}),
  ToolMessage(conten

In [25]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Удали из корзины Хлеб пшеничный из муки 1 сорта, 200 грамм"}]},
    context={"user_role": "expert"}
)
result

{'messages': [HumanMessage(content='Удали из корзины Хлеб пшеничный из муки 1 сорта, 200 грамм', additional_kwargs={}, response_metadata={}, id='9f16b210-33b8-4a03-969e-19d3c5db940e'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 1566, 'total_tokens': 1611, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 1520}}, 'model_provider': 'openai', 'model_name': 'Qwen/Qwen3-235B-A22B-Instruct-2507', 'system_fingerprint': None, 'id': 'chatcmpl-313fd071-791f-4557-9389-c248c22c8530', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--96b41a58-8622-4233-a5c8-8eb8f49981ac-0', tool_calls=[{'name': 'remove_from_cart', 'args': {'product_name': 'Хлеб пшеничный из муки 1 сорта', 'amount': 200}, 'id': 'chatcmpl-tool-4318c8eab64b4c8ca02546c089d65479', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1566, 'output_tokens': 45, 'total_tokens': 1611,

In [26]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Удали из корзины Ветчина рубленая"}]},
    context={"user_role": "expert"}
)
result

{'messages': [HumanMessage(content='Удали из корзины Ветчина рубленая', additional_kwargs={}, response_metadata={}, id='f886061e-2ce5-4abd-b8a7-a490702287ad'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 1549, 'total_tokens': 1577, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 1536}}, 'model_provider': 'openai', 'model_name': 'Qwen/Qwen3-235B-A22B-Instruct-2507', 'system_fingerprint': None, 'id': 'chatcmpl-e0e078e6-f400-471f-91e1-26607efa7e91', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--1d01a18e-d4ce-4b10-b9a5-bd64c9183a58-0', tool_calls=[{'name': 'remove_from_cart', 'args': {'product_name': 'Ветчина рубленая'}, 'id': 'chatcmpl-tool-652f09ffee46401b87e9bd41c5d211c1', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1549, 'output_tokens': 28, 'total_tokens': 1577, 'input_token_details': {'cache_read': 1536}, 'output_

In [27]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Покажи, что в корзине"}]},
    context={"user_role": "expert"}
)
result

{'messages': [HumanMessage(content='Покажи, что в корзине', additional_kwargs={}, response_metadata={}, id='52d461fb-0060-4f38-91db-49ba120ff22f'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 1546, 'total_tokens': 1564, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 1536}}, 'model_provider': 'openai', 'model_name': 'Qwen/Qwen3-235B-A22B-Instruct-2507', 'system_fingerprint': None, 'id': 'chatcmpl-851b3dea-bb5b-4385-a8de-c16875be29c0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--e8c8af26-d2bd-426f-b36d-24080cf9768a-0', tool_calls=[{'name': 'show_product_in_cart', 'args': {}, 'id': 'chatcmpl-tool-7e509d8891ed4fff80666878d248d56a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1546, 'output_tokens': 18, 'total_tokens': 1564, 'input_token_details': {'cache_read': 1536}, 'output_token_details': {}}),
  ToolMessage(conten